In [12]:
using LinearAlgebra, IntervalArithmetic, ForwardDiff

# 候補区間Xを定義
X = [(-0.4.. -0.3),(-0.3.. -0.2),
    (-0.3.. -0.2),(-0.2.. -0.1),
    (-0.1.. -0.0),(-0.1.. -0.0),
    (0.0..0.01),(2.0..3.0),
    (2.0..3.0),(2.0..3.0)]

# 計算対象となる方程式を定義
g(x) = 2.5x^3-10.5x^2+11.8x

f1(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10) = g(x1)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-1
f2(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x2)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-2
f3(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x3)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-3
f4(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x4)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-4
f5(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x5)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-5
f6(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x6)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-6
f7(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x7)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-7
f8(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x8)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-8
f9(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x9)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-9
f10(x1, x2, x3, x4, x5, x6,x7,x8,x9,x10) = g(x10)+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10-10

F( (x1,x2,x3,x4,x5,x6,x7,x8,x9,x10) ) = [f1(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10); f2(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);
    f3(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);f4(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);
    f5(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);f6(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);
    f7(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);f8(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);
    f9(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);f10(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10);]

Df((x1,x2,x3,x4,x5,x6,x7,x8,x9,x10) ) = [7.5x1^2-21x1^2+12.8 1 1 1 1 1 1 1 1 1;
1 7.5x2^2-21x2^2+12.8 1 1 1 1 1 1 1 1;
1 1 7.5x3^2-21x3^2+12.8 1 1 1 1 1 1 1;
1 1 1 7.5x4^2-21x4^2+12.8 1 1 1 1 1 1;
1 1 1 1 7.5x5^2-21x5^2+12.8 1 1 1 1 1;
1 1 1 1 1 7.5x6^2-21x6^2+12.8 1 1 1 1;
1 1 1 1 1 1 7.5x7^2-21x7^2+12.8 1 1 1;
1 1 1 1 1 1 1 7.5x8^2-21x8^2+12.8 1 1;
1 1 1 1 1 1 1 1 7.5x9^2-21x9^2+12.8 1;
1 1 1 1 1 1 1 1 1 7.5x9^2-21x9^2+12.8;]

LoadError: cannot define function Df; it already has a value

In [6]:
#ニュートン法で近似解を計算する
function newton(F,x0)
    #初期値を設定
    tol = 5e-10; count = 0;
    x = x0;
    Fx = F(x);
    println("Before iteration: $(Fx)")
    #条件の範囲内で計算を回す
    while maximum(abs.(Fx)) ≥ tol && count ≤ 20
        DF = ForwardDiff.jacobian(F,x);
        x -= DF\Fx;
        Fx = F(x);
        println("After $(count) th iteration: $(Fx)")
        count += 1;
    end
    return x
end

#クラフチック法を計算する
function krawczyk(F,X)
    iDF = ForwardDiff.jacobian(F,X);
    c = mid.(X); ic = map(Interval,c);
    DF = ForwardDiff.jacobian(F,c);
    R = inv(DF);
    M = Matrix{Float64}(I,size(R)) - R*iDF;
    #クラフチック写像の値を返す
    return c - R*F(ic) + M*(X - c)
end

#最終的に完成した関数
function verifynlss_krawczyk(F,c)
    DF = ForwardDiff.jacobian(F,c)
    R = inv(DF)
    r = abs.(R*F(c))
    u = r .+ (sum(r)/length(r))
    X = c .± u
    K = krawczyk(F,X)
    #範囲内に入っていたら、さらに範囲の精度をあげていく
    if all(K .⊂ X)
        tol = 5e-10
        count = 0
        while maximum(radius.(K)) >= tol && count ≤ 100
            K = krawczyk(F,K)
            count += 1
            radius.(K)
        end
        success = 1
        return success, K
    end
    println("Oh my way, verification is failed...return a improved approximate solution") # cをNewton法で改善しても良い。
    success = 0
    return success, newton(F,c)
end

# success, X = verifynlss_krawczyk(F,[-0.33,-0.28,-0.22,-0.16,-0.09,-0.01,0.06,2.20,2.47,2.61])
# if success == 0
#     success, X = verifynlss_krawczyk(F,X)
# end
# success, X

verifynlss_krawczyk (generic function with 1 method)

In [7]:
#近似解を計算
t = newton(F,mid.(X))

Before iteration: [-0.11843749999999975, 0.7596874999999992, -0.24031250000000082, 0.3903125000000003, 0.7884374999999997, -0.21156250000000032, -0.5362621875000002, 1.3424999999999994, 0.34249999999999936, -0.6575000000000006]
After 0 th iteration: [-0.0016944178177271674, -0.017155847544205738, -0.004923587845226063, -0.004016883441444019, -0.03060706242885214, -0.006744324864918383, -0.03150971377042744, 0.30886698543880975, 0.011578659047541962, 0.1321911589136402]
After 1 th iteration: [-9.468522982691852e-5, -0.0001756280435714075, -0.00013461081700238964, -0.00015030586739328555, -0.0003860443819441173, -0.00023900273781407577, -0.000620870843539123, 0.042872429654563504, 0.0004130783268223581, 0.000854973207157883]
After 2 th iteration: [-2.7462843172898488e-6, -3.123800929039078e-6, -3.508907373550585e-6, -4.0653362534115445e-6, -5.064575166358054e-6, -6.0182375651862685e-6, -8.457664053551639e-6, 0.001538935163766908, 1.8151653890186026e-5, 8.966373354013513e-6]
After 3 th it

10-element Array{Float64,1}:
 -0.3354530832297739
 -0.2828512314544382
 -0.2261366063545622
 -0.164364623112089
 -0.09617922680493231
 -0.019512906781618936
  0.06906744069594639
  2.202988195425023
  2.4710752069830084
  2.615635620753133

In [35]:
using BenchmarkTools
#クラフチックでの時間測定
@btime verifynlss_krawczyk($F,$t)

  294.917 μs (1648 allocations: 97.88 KiB)


(1, Interval{Float64}[[-0.335454, -0.335453], [-0.282852, -0.282851], [-0.226137, -0.226136], [-0.164365, -0.164364], [-0.0961793, -0.0961792], [-0.019513, -0.0195129], [0.0690674, 0.0690675], [2.20298, 2.20299], [2.47107, 2.47108], [2.61563, 2.61564]])

In [39]:
include("IntervalLinearAlgebra.jl");

#区間連立1次方程式を解く関数
function verifylss_iAib(iA,ib) 
    A = mid.(iA)
    b = mid.(ib)
    x̄ = A\b
    n = length(x̄)
    R = inv(A)
    #########
    G = Matrix{Float64}(I, n, n) - R*iA
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄)
        r = iA*x̄ - ib # Interval arithmetic
        Rr = R*r
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)) # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan
    end
    return x̄ .± supremum.(err)
end

verifylss_iAib (generic function with 1 method)

In [40]:
#ニュートン法を計算
function newton(F,x0)
    #初期値を設定
    tol = 5e-10; count = 0;
    x = x0;
    Fx = F(x);
    #条件によってニュートン法をまわす
    while maximum(abs.(Fx)) ≥ tol && count ≤ 20
        DF = ForwardDiff.jacobian(F,x);
        x -= DF\Fx;
        Fx = F(x);
        count += 1;
    end
    return x
end

#N(c,X)を計算する関数
function IntervalNewton(F,X)
    c = mid.(X);
    ic = map(Interval,c);
    M = ForwardDiff.jacobian(F,X);
    b = F(ic);
    #N(c,X)の値を返す
    return ic - verifylss_iAib(M,b)
    #return ic - M\b
end

#最終的に構築した関数
function verifynlss_IntervalNewton(F, c)
    DF = ForwardDiff.jacobian(F,c);
    R = inv(DF);
    r = abs.(R*F(c));
    u = r .+ (sum(r)/length(r));
    X = c .± u;
    K = IntervalNewton(F,X);
    #範囲内に入っていたら、さらに解の精度をあげる
    if all(K .⊂ X)
        tol = 5e-10;
        while maximum(radius.(K)) >= tol && count ≤ 100
            K = IntervalNewton(F,K)
        end
        success = 1
        return success, K
    end
    println("Oh my way, verification is failed...return a improved approximate solution") 
    success = 0
    return success, newton(F,c)
end

verifynlss_IntervalNewton (generic function with 1 method)

In [41]:
#区間ニュートンでの時間測定
@btime verifynlss_IntervalNewton($F,$t)

  320.791 μs (1665 allocations: 99.75 KiB)


(1, Interval{Float64}[[-0.335454, -0.335453], [-0.282852, -0.282851], [-0.226137, -0.226136], [-0.164365, -0.164364], [-0.0961793, -0.0961792], [-0.019513, -0.0195129], [0.0690674, 0.0690675], [2.20298, 2.20299], [2.47107, 2.47108], [2.61563, 2.61564]])